# Table Testing Automation Script

### Initialisation

In [14]:
import time
import sqlite3
import numpy as np
import pandas as pd

### Import queries, schema info & clean entries for execution, set number formatting
queryDocPath = "/Users/wayne/Desktop/SQL/queryDoc.xlsx"
pd.set_option('display.float_format', lambda x: '%.2f' % x)

def schemaInfoImport(filepath):
    importDataframe = pd.read_excel(filepath, sheet_name="query-jupyter", header=0)
    global sql_list
    global table_list
    global group_by_list
    sql_list        = importDataframe['QUERY'].tolist()
    table_list      = importDataframe['Table Name'].tolist()
    group_by_list   = importDataframe['Group By'].tolist()

def attributeCleaner(groupbyList):
    for m in range(len(groupbyList)):
        group_by_list[m] = group_by_list[m].split(',')
        for n in range(len(groupbyList[m])):
            groupbyList[m][n] = groupbyList[m][n].lstrip(' ')
            groupbyList[m][n] = groupbyList[m][n].replace(u'\xa0', u'')
            
schemaInfoImport(queryDocPath)            
attributeCleaner(group_by_list)

print('Example query:',sql_list[0])

Example query: SELECT entity_identifier, group_by1, sum(balance_in_ccy1) as balance_in_ccy1, sum(balance_in_ccy2) as balance_in_ccy2, count(*) as COUNT FROM FINOTC_LCR_SRC_DATA1 GROUP BY entity_identifier, group_by1


### Execution

In [15]:
start = time.time()

# Creates database connection
con = sqlite3.connect("/Users/wayne/DataGripProjects/FIN_DATA/FIN_DATABASE.sqlite")

### Create Table object
class Table:
    
    # Initialise object
    def __init__(self, name, query, groupby, status):
        self.name = name
        self.query = query
        self.groupby = groupby
        self.status = {}
        print('Table:',self.name)
        
    ### Import tables from both environments       
    def ENV1_import(self):
        global ENV1_raw
        ENV1_raw = pd.read_sql_query(self.query, con)
        ENV1_pivot = pd.pivot_table(ENV1_raw,index=self.groupby,  values = (ENV1_raw.columns).tolist(), aggfunc='sum', fill_value=0)
        return(ENV1_pivot)
    
    def ENV2_import(self):
        ENV2_filepath = ("/Users/wayne/Desktop/SQL/csv/"+ self.name + ".csv")
        global ENV2_raw
        ENV2_raw = pd.read_csv(ENV2_filepath, header=0)
        ENV2_pivot = pd.pivot_table(ENV2_raw,index=self.groupby, values = (ENV2_raw.columns).tolist(), aggfunc='sum')
        return(ENV2_pivot)
        
    ### Perform comparison and return absolute & relative variances
    def ENV_comparison(self):
        absolute_variances = self.ENV2_import().subtract(self.ENV1_import(), fill_value=0)
        relative_variances = (absolute_variances.div(self.ENV1_import()[self.ENV1_import().columns.tolist()], fill_value=0)).fillna(0)
        relative_variances = relative_variances.add_prefix('% Variance in ')
        relative_variances = (relative_variances.multiply(100)).replace(np.inf, 100)
        
        final_variance_table = pd.concat([absolute_variances, relative_variances], axis=1)
        return(final_variance_table)
    
    ### Calculate grand total variances for each column
    def grandTotalfunc(self):
        ENV1_grandTotals = self.ENV1_import().append(ENV1_raw[self.ENV1_import().columns].sum().rename('Grand Total')).fillna('')
        ENV2_geandTotals = self.ENV2_import().append(ENV2_raw[self.ENV2_import().columns].sum().rename('Grand Total')).fillna('')
        
        GT_variances = ((ENV1_grandTotals.tail(1)-ENV2_geandTotals.tail(1)) / ENV1_grandTotals.tail(1)).multiply(100)
        GT_variances = GT_variances.add_prefix('% Variance in ')
        
        final_variance_table_TOTALS = pd.concat([(ENV1_grandTotals.tail(1)-ENV2_geandTotals.tail(1)), GT_variances], axis=1).fillna(0)
        final_variance_table_TOTALS.replace([np.inf,-np.inf], 0 , inplace=True) 
        return(final_variance_table_TOTALS)
    
    ### Determines Pass/Fail according to test types & data
    def statusChecker(self, tablename, row, rowval, balance, balval, colname, colcount, details):
        self.status[tablename] = self.name
        
        # Row Count
        if abs((self.grandTotalfunc()['COUNT'].sum())) > 0: self.status[row] = ("Fail")
        if abs((self.grandTotalfunc()['COUNT'].sum())) == 0: self.status[row] = ("Pass")
        self.status[rowval] = ((self.grandTotalfunc())['COUNT'].sum())
        
        # Balances
        if abs(((self.grandTotalfunc().drop(['COUNT'], axis=1)).values.sum())) < 5: self.status[balance] = ("Pass")
        if abs(((self.grandTotalfunc().drop(['COUNT'], axis=1)).values.sum())) > 5: self.status[balance] = ("Fail")
        self.status[balval] = (((self.grandTotalfunc()).drop(['COUNT'], axis=1)).values.sum())
        
        # Column Count
        if len(ENV2_raw.columns) != len(ENV1_raw.columns): self.status[colcount] = ("Fail")
        if len(ENV2_raw.columns) == len(ENV1_raw.columns): self.status[colcount] = ("Pass")
        
        # Column Names
        if ENV1_raw.columns.tolist() != ENV2_raw.columns.tolist(): self.status[colname] = ("Fail")
        if ENV1_raw.columns.tolist() == ENV2_raw.columns.tolist(): self.status[colname] = ("Pass")
        
        # Generates variance descriptions
        templist = []
        for name, values in self.grandTotalfunc().iteritems():
            temp = ('{name}: {value}'.format(name=name, value=values[0]))
            templist.append(temp)
        self.status[details] = templist
        
        return self.status
    
    ### Generates xlsx for each table with raw data, pivot data and comparison data   
    def excelGenerator(self):    
        with pd.ExcelWriter(r'/Users/wayne/Desktop/SQL/Results/' + self.name + "_VARIANCE.xlsx") as writer: 
            ENV1_raw.to_excel(writer, sheet_name='ENV1 raw data')
            self.ENV1_import().to_excel(writer, sheet_name='ENV1 Pivot')
            ENV2_raw.to_excel(writer, sheet_name='ENV2 raw data')    
            self.ENV2_import().to_excel(writer, sheet_name='ENV2 Pivot')
            self.ENV_comparison().to_excel(writer, sheet_name='Variances')

### Executes functions iteratively for every tables   
results_data_list = []    
defect_df_names   = []
defect_df_details = []
for x in range(len(table_list)):
    table = Table(table_list[x], sql_list[x], group_by_list[x], status={})
    table.ENV1_import()
    table.ENV2_import()
    
    # Displays comparison data for first table for observation
    if x == 0:
        display(table.ENV_comparison())
        display(table.grandTotalfunc())
    else:
        table.ENV_comparison()
        table.grandTotalfunc()
        
    # Creates dictionary for each table to store data
    results_data = table.statusChecker('Table Name','Row Count', 'Row Count Value', 'Balance', 'Balance Value', 'Col Name', 'Col Count', 'Details')
    results_data_list.append(results_data)
    
    # Generates data format for defect_descriptions_df
    for item in range(len(results_data['Details'])):
        defect_df_names.append(results_data['Table Name'])
        defect_df_details.append(results_data['Details'][item])
    
    table.excelGenerator()
    
defect_description_df = pd.DataFrame(data=zip(defect_df_names, defect_df_details), columns = ['Table Name', 'Details'])
display(defect_description_df)

results_df = (pd.DataFrame(results_data_list))
results_df = results_df.drop('Details', axis=1)
display(results_df)

# Closes database connection
con.close() 

# Generates high level excel dashboard
with pd.ExcelWriter(r'/Users/wayne/Desktop/SQL/Results/'+'Dashboards.xlsx') as writer: 
        results_df.to_excel(writer, sheet_name='Results')
        defect_description_df.to_excel(writer, sheet_name='Descriptions')

# Prints execution time
end = time.time()
exec_time_seconds = end - start
print('Execution time: ' + str(exec_time_seconds) + ' seconds.')


Table: FINOTC_LCR_SRC_DATA1


COUNT  balance_in_ccy1  balance_in_ccy2  \
entity_identifier group_by1                                               
231               GER         4689.00          5422.00             3.40   
3600              ENG           64.00             0.00             0.00   
                  SWE       901652.00     -97089201.00            -0.00   
8383              ENG          974.00         53683.00             0.00   
                  FRA          974.00       -674928.00             0.00   
93720             AUS        35255.00        432432.00         34332.43   

                             % Variance in COUNT  \
entity_identifier group_by1                        
231               GER                     100.00   
3600              ENG                    6400.00   
                  SWE                90165200.00   
8383              ENG                   97400.00   
                  FRA                   97400.00   
93720             AUS                     100.00   

                             % Variance in balance_in_ccy1  \
entity_identifier group_by1                                  
231               GER                               100.00   
3600              ENG                                 0.00   
                  SWE                               -99.96   
8383              ENG                              1283.67   
                  FRA                               -99.92   
93720             AUS                               100.00   

                             % Variance in balance_in_ccy2  
entity_identifier group_by1                                 
231               GER                               100.00  
3600              ENG                                 0.00  
                  SWE                                -0.00  
8383              ENG                                 0.00  
                  FRA                                 0.00  
93720             AUS                               100.00

,COUNT,balance_in_ccy1,balance_in_ccy2,% Variance in COUNT,% Variance in balance_in_ccy1,% Variance in balance_in_ccy2
Grand Total,-943608.00,97272592.00,-34335.83,-23590200.00,98.98,-0.78


Table: FINOTC_LCR_REP_DATA1
Table: RWA_REPORTING_INTER_DATA
Table: RWA_PRA_DATA
Table: LIQ_PRA_DATA
Table: CRM_BB_PRODUCT_DATA


,Table Name,Details
0,FINOTC_LCR_SRC_DATA1,COUNT: -943608.0
1,FINOTC_LCR_SRC_DATA1,balance_in_ccy1: 97272592.0
2,FINOTC_LCR_SRC_DATA1,balance_in_ccy2: -34335.830299999565
3,FINOTC_LCR_SRC_DATA1,% Variance in COUNT: -23590200.0
4,FINOTC_LCR_SRC_DATA1,% Variance in balance_in_ccy1: 98.98007953565528
5,FINOTC_LCR_SRC_DATA1,% Variance in balance_in_ccy2: -0.781753768984...
6,FINOTC_LCR_REP_DATA1,COUNT: -7558.0
7,FINOTC_LCR_REP_DATA1,balance: 915719.0
8,FINOTC_LCR_REP_DATA1,balance2: 7837.569880000001
9,FINOTC_LCR_REP_DATA1,% Variance in COUNT: -151160.0


,Table Name,Row Count,Row Count Value,Balance,Balance Value,Col Count,Col Name
0,FINOTC_LCR_SRC_DATA1,Fail,-943608.00,Fail,73648154.37,Fail,Fail
1,FINOTC_LCR_REP_DATA1,Fail,-7558.00,Fail,772515.45,Fail,Fail
2,RWA_REPORTING_INTER_DATA,Fail,-109229.00,Fail,-2184580.00,Fail,Fail
3,RWA_PRA_DATA,Fail,-6337.00,Fail,-126740.02,Fail,Fail
4,LIQ_PRA_DATA,Fail,-2350.00,Fail,9200534.27,Fail,Fail
5,CRM_BB_PRODUCT_DATA,Pass,0.00,Pass,-0.01,Pass,Pass


Execution time: 1.2012779712677002 seconds.
